# AUTOGRAD: AUTOMATIC DIFFERENTIATION

在PyTorch中，神经网络运算的核心就是`autograd`包。 首先我们来看一下这个包是怎么工作的，然后来创建我们的第一个神经网络。

`autograd`包能够为所有张量运算进行微分，也就是求导。PyTorch中的网络结构是根据代码的运行来定义的(**It is a define-by-run framework**)，也就是说，反向传播的过程是根据代码运行的过程来定义的，因此每一轮迭代都可以不同。

下面来看例子。

# Tensor

`torch.Tensor` 是PyTorch的核心，如果你它的属性`.requires_grad`设置为`True`，框架就会开始在这个张量上进行的所有运算。当完成计算时，你可以调用`.backward()`函数来自动计算出所有的导数。当前tensor的导数会保存在`.grad`属性中。

如果你不想计算某个张量的导数，你可以调用其`.detach()`方法。

你也可用`with torch.no_grad():`来包含无需计算导数的代码块，例如在对模型进行评估的时候，模型中有些参数是可以被训练的(`requires_grad=True`)，而此时我们并不需要其导数。

另外还有一个很重要的类就是`Function`类，`Tensor`和`Function`相互连通构成了一个无环图，这个无环图记载了整个计算过程。每个tensor都有一个`.grad_fn`属性，这个属性指向创建这个`Tensor`的`Function`。由用户创建的张量，此属性为`None`。

如果你想计算偏导数，可以调用张量的`.backward`方法。如果张量是一个标量(例如保存了一个数值)，那么你无需提供参数。如果是一个向量，那么你需要指定`.backward`方法的`gradient`参数，且参数的形状需和tensor的形状相同。

In [1]:
import torch

创建一个张量，并跟踪在它上面进行的计算：

In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


进行张量运算：

In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


`y`由运算生成，所以它有一个`grad_fn`属性。

In [4]:
print(y.grad_fn)

再进行几个运算：

In [5]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward1>)


方法`.requires_grad_(...)`能够修改张量的`requires_grad`属性，函数的默认参数为`False`。

In [6]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


# 梯度

现在我们可以进行反向传播。因为`out`只包含了一个标量，所以`out.backward()`等价于`out.backward(torch.tensor(1))`。

In [7]:
out.backward()

输出偏导数$\frac{d(out)}{dx}$

In [8]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


下面来看下计算过程（$o$代表`out`）：
$$o = \frac{1}{4} \sum_{i=1}^{i=4} z_i, z_i=3(x_i+2)^2$$
所以：
$$\frac{\partial o}{\partial x_i}=\frac{3}{2}(x_i+2)$$
那么：
$$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2}=4.5$$

在数学上，如果你有一个向量化的函数：$\vec{y}=f(\vec{x})$，那么向量 $\vec{y}$ 关于向量 $\vec{x}$ 的导数为一个雅可比矩阵(Jacobian matrix)：
$$
\begin{split}J=\left(\begin{array}{ccc}
 \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
 \vdots & \ddots & \vdots\\
 \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
 \end{array}\right)\end{split}
$$

通俗地讲，`torch.autograd`就是一个计算向量-雅可比点积(vector-Jacobian product)的引擎。假设$\vec{v}$是一个标量函数$l=g(\vec{y})$的导数，即$v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$，那么根据链式法则，那么$l$关于$x$的偏导数则为：
$$
\begin{split}J^{T}\cdot v=\left(\begin{array}{ccc}
 \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
 \vdots & \ddots & \vdots\\
 \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
 \end{array}\right)\left(\begin{array}{c}
 \frac{\partial l}{\partial y_{1}}\\
 \vdots\\
 \frac{\partial l}{\partial y_{m}}
 \end{array}\right)=\left(\begin{array}{c}
 \frac{\partial l}{\partial x_{1}}\\
 \vdots\\
 \frac{\partial l}{\partial x_{n}}
 \end{array}\right)\end{split}
$$
这种向量-雅可比计算方法使得对**具有向量输出的模型**反向计算梯度变得非常方便。下面来看个例子：

In [14]:
x = torch.randn(3, requires_grad=True)

y = x * x
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([1042.3011,  740.9583,   35.3033], grad_fn=<MulBackward0>)


现在 $y$ 不是一个标量，`torch.autograd`不能计算整个雅可比矩阵，但是如果我们传入一个向量作为`backward`函数的参数，那么我们就能得到向量-雅可比点积：

In [15]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)
print(x.grad)

tensor([-2.9221e+02, -2.4637e+03,  5.3778e-02])


我们可以将代码块放在`with torch.no_grad():`中，以停止追踪计算历史（即便设置了`.requires_grad=True`也可以）。

In [17]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad)

True
True
False
